In [1]:
import string
import re
import random
import collections
import copy
import os

from IPython.display import clear_output

from enum import Enum
Direction = Enum('Direction', 'left right up down upleft upright downleft downright')
    
delta = {Direction.left: (0, -1),Direction.right: (0, 1), 
         Direction.up: (-1, 0), Direction.down: (1, 0), 
         Direction.upleft: (-1, -1), Direction.upright: (-1, 1), 
         Direction.downleft: (1, -1), Direction.downright: (1, 1)}

cat = ''.join
wcat = ' '.join
lcat = '\n'.join

In [180]:
# all_words = [w.strip() for w in open('/usr/share/dict/british-english').readlines()
#             if all(c in string.ascii_lowercase for c in w.strip())]
# words = [w for w in all_words
#          if not any(w in w2 for w2 in all_words if w != w2)]
# open('wordsearch-words', 'w').write(lcat(words))

In [181]:
# ws_words = [w.strip() for w in open('wordsearch-words').readlines()
#             if all(c in string.ascii_lowercase for c in w.strip())]
# ws_words[:10]

In [2]:
ws_words = [w.strip() for w in open('/usr/share/dict/british-english').readlines()
            if all(c in string.ascii_lowercase for c in w.strip())]

In [3]:
len([w for w in ws_words if len(w) >= 4])

62183

In [4]:
def empty_grid(w, h):
    return [['.' for c in range(w)] for r in range(h)]

In [5]:
def show_grid(grid):
    return lcat(cat(r) for r in grid)

In [6]:
grid = empty_grid(10, 10)
print(show_grid(grid))

..........
..........
..........
..........
..........
..........
..........
..........
..........
..........


In [7]:
def indices(grid, r, c, l, d):
    dr, dc = delta[d]
    w = len(grid[0])
    h = len(grid)
    inds = [(r + i * dr, c + i * dc) for i in range(l)]
    return [(i, j) for i, j in inds
           if i >= 0
           if j >= 0
           if i < h
           if j < w]

In [8]:
def gslice(grid, r, c, l, d):
    return [grid[i][j] for i, j in indices(grid, r, c, l, d)]

In [9]:
def set_grid(grid, r, c, d, word):
    for (i, j), l in zip(indices(grid, r, c, len(word), d), word):
        grid[i][j] = l
    return grid

In [10]:
set_grid(grid, 2, 3, Direction.downright, 'testword')
print(show_grid(grid))

..........
..........
...t......
....e.....
.....s....
......t...
.......w..
........o.
.........r
..........


In [191]:
cat(gslice(grid, 3, 2, 15, Direction.right))

'..e.....'

In [192]:
re.match(cat(gslice(grid, 3, 2, 4, Direction.right)), 'keen')

<_sre.SRE_Match object; span=(0, 4), match='keen'>

In [193]:
re.match(cat(gslice(grid, 3, 2, 3, Direction.right)), 'keen')

<_sre.SRE_Match object; span=(0, 3), match='kee'>

In [194]:
re.fullmatch(cat(gslice(grid, 3, 2, 3, Direction.right)), 'keen')

In [195]:
re.match(cat(gslice(grid, 3, 2, 4, Direction.right)), 'kine')

In [11]:
def could_add(grid, r, c, d, word):
    s = gslice(grid, r, c, len(word), d)
    return re.fullmatch(cat(s), word)

In [197]:
could_add(grid, 3, 2, Direction.right, 'keen')

<_sre.SRE_Match object; span=(0, 4), match='keen'>

In [198]:
could_add(grid, 3, 2, Direction.right, 'kine')

In [199]:
random.choice(list(Direction))

<Direction.up: 3>

In [14]:
def present(grid, word):
    w = len(grid[0])
    h = len(grid)
    for r in range(h):
        for c in range(w):
            for d in Direction:
                if cat(gslice(grid, r, c, len(word), d)) == word:
                    return True, r, c, d
    return False, 0, 0, list(Direction)[0]

In [15]:
def present_many(grid, words):
    w = len(grid[0])
    h = len(grid)
    wordlens = set(len(w) for w in words)
    presences = []
    for r in range(h):
        for c in range(w):
            for d in Direction:
                for wordlen in wordlens:
                    word = cat(gslice(grid, r, c, wordlen, d))
                    if word in words:
                        presences += [(word, r, c, d)]
    return set(presences)

In [16]:
def fill_grid(grid, words, word_count, max_attempts=10000, min_word_len=4):
    attempts = 0
    added_words = []
    w = len(grid[0])
    h = len(grid)
    while len(added_words) < word_count and attempts < max_attempts:
        clear_output(wait=True)
        print("Added", len(added_words), '; attempt', attempts)
        attempts += 1
        r = random.randrange(w)
        c = random.randrange(h)
        word = random.choice(words)
        d = random.choice(list(Direction))
#         print(word, r, c, d, 
#               len(word) >= min_word_len, 
#               not any(word in w2 for w2 in added_words), 
#               could_add(grid, r, c, d, word), 
#               not present(grid, word)[0]
#              )
        if (len(word) >= min_word_len
              and not any(word in w2 for w2 in added_words) 
              and could_add(grid, r, c, d, word)
              and not present(grid, word)[0]):
            grid2 = copy.deepcopy(grid)
            set_grid(grid2, r, c, d, word)
#             print(present_many(grid2, [word]))
            if collections.Counter(p[0] for p in 
                                   present_many(grid2, added_words + [word])).most_common(1)[0][1] == 1:
                set_grid(grid, r, c, d, word)
                added_words += [word]
                attempts = 0
    return grid, added_words

In [17]:
g = empty_grid(20, 20)
g, ws = fill_grid(g, ws_words, 40)
len(ws)

Added 39 ; attempt 9


40

In [18]:
print(show_grid(g))
print(len(ws), 'words added')
print(wcat(ws))

emityad..yltnetap...
noitcesib.selbmur.b.
....hy...dc.......a.
..g.et.r.eoretset.s.
p.rwri.apkud......k.
udoonu.vaene......sl
ritmir.elett.....me.
iseaaf.lspei...taaru
tssne.teatrd..esp.en
aiqlcoifedbe.uhs..dh
nmui.s.yc.ansi.e.nae
suee.g.bnplunskce.re
.lss.n.raeag.iemh.gd
.ast.i.ahan.r.er.u.e
.t.f.l.enhct.ra.a.nd
.e..fs.neei.og.l.wik
.d...u...nnf..e.pde.
elppa.n..sg....eya.b
.......s.waist.ls.c.
suppression.vilere.a
40 words added
smashing counterbalancing womanliest teat unedited unheeded puritans apple patently dissimulated bask tester bewares suet herniae nearby idyll geese grader waist grotesques slap peahens ravel suppression rumbles snuffs peeked bisection alpaca enhance daytime trike leaps coifed foremen chunk slings viler fruity


In [ ]:
for w in ws:
    print(w, present(g, w))

In [19]:
def interesting(grid, words, words_limit=40, direction_slack=1):
    dirs = set(present(grid, w)[3] for w in words)
    return len(words) > words_limit and len(dirs) + direction_slack >= len(delta)

In [ ]:
interesting(g, ws)

In [20]:
def interesting_grid(width=20, height=20, words_limit=40, max_words=80, direction_slack=1):
    boring = True
    while boring:
        grid = empty_grid(width, height)
        grid, words = fill_grid(grid, ws_words, max_words)
        boring = not interesting(grid, words, words_limit=words_limit, direction_slack=direction_slack)
    return grid, words

In [ ]:
g, ws = interesting_grid()
print(show_grid(g))
print(len(ws), 'words added; ', len(set(present(g, w)[3] for w in ws)), 'directions')
print(wcat(ws))

In [21]:
def datafile(name, sep='\t'):
    """Read key,value pairs from file.
    """
    with open(name) as f:
        for line in f:
            splits = line.split(sep)
            yield [splits[0], int(splits[1])]

In [22]:
def normalise(frequencies):
    """Scale a set of frequencies so they sum to one
    
    >>> sorted(normalise({1: 1, 2: 0}).items())
    [(1, 1.0), (2, 0.0)]
    >>> sorted(normalise({1: 1, 2: 1}).items())
    [(1, 0.5), (2, 0.5)]
    >>> sorted(normalise({1: 1, 2: 1, 3: 1}).items()) # doctest: +ELLIPSIS
    [(1, 0.333...), (2, 0.333...), (3, 0.333...)]
    >>> sorted(normalise({1: 1, 2: 2, 3: 1}).items())
    [(1, 0.25), (2, 0.5), (3, 0.25)]
    """
    length = sum(f for f in frequencies.values())
    return collections.defaultdict(int, ((k, v / length) 
        for (k, v) in frequencies.items()))


In [23]:
english_counts = collections.Counter(dict(datafile('count_1l.txt')))
normalised_english_counts = normalise(english_counts)

In [24]:
wordsearch_counts = collections.Counter(cat(ws_words))
normalised_wordsearch_counts = normalise(wordsearch_counts)

In [25]:
normalised_wordsearch_counts = normalise(collections.Counter(normalised_wordsearch_counts) + collections.Counter({l: 0.05 for l in string.ascii_lowercase}))

In [35]:
def weighted_choice(d):
    """Generate random item from a dictionary of item counts
    """
    target = random.uniform(0, sum(d.values()))
    cuml = 0.0
    for (l, p) in d.items():
        cuml += p
        if cuml > target:
            return l
    return None

def random_english_letter():
    """Generate a random letter based on English letter counts
    """
    return weighted_choice(normalised_english_counts)

def random_wordsearch_letter():
    """Generate a random letter based on wordsearch letter counts
    """
    return weighted_choice(normalised_wordsearch_counts)

In [ ]:
cat(sorted(random_english_letter() for i in range(100)))

In [ ]:
cat(sorted(random_wordsearch_letter() for i in range(100)))

In [37]:
random_wordsearch_letter()

'h'

In [27]:
def pad_grid(g0):
    grid = copy.deepcopy(g0)
    w = len(grid[0])
    h = len(grid)
    for r in range(h):
        for c in range(w):
            if grid[r][c] == '.':
                grid[r][c] = random_wordsearch_letter()
    return grid

In [ ]:
padded = pad_grid(g)
print(show_grid(padded))

In [ ]:
print(show_grid(g))

In [ ]:
for w in ws:
    print(w, present(padded, w))

In [28]:
def decoys(grid, words, all_words, limit=100):
    decoy_words = []
    dlen_limit = max(len(w) for w in words)
    while len(words) + len(decoy_words) < limit:
        d = random.choice(all_words)
        if d not in words and len(d) >= 4 and len(d) <= dlen_limit and not present(grid, d)[0]:
            decoy_words += [d]
    return decoy_words

In [55]:
def decoys(grid, words, all_words, limit=100, min_word_len=4):
    decoy_words = []
#     dlen_limit = max(len(w) for w in words)
    while len(words) + len(decoy_words) < limit:
        d = random.choice(all_words)
        if (d not in words and 
            d not in decoy_words and 
            len(d) >= min_word_len and 
            not any(w in d for w in (decoy_words + words)) and
            not any(d in w for w in decoy_words) and
            not present(grid, d)[0]):
            decoy_words += [d]
    return decoy_words

In [ ]:
ds = decoys(padded, ws, ws_words)
ds

In [ ]:
for w in ws + ds:
    print(w, present(padded, w))

In [ ]:
g, ws = interesting_grid()
p = pad_grid(g)
ds = decoys(p, ws, ws_words)
print(show_grid(g))
print(show_grid(p))
print(len(ws), 'words added; ', len(set(present(g, w)[3] for w in ws)), 'directions')
print('Present:', wcat(sorted(ws)))
print('Decoys:', wcat(sorted(ds)))

In [ ]:
g, ws = interesting_grid(width=10, height=10, words_limit=5, direction_slack=6)
p = pad_grid(g)
ds = decoys(p, ws, ws_words)
print(show_grid(g))
print()
print(show_grid(p))
print(len(ws), 'words added; ', len(set(present(g, w)[3] for w in ws)), 'directions')
print('Present:', wcat(sorted(ws)))
print('Decoys:', wcat(sorted(ds)))

In [ ]:
ds_original = ['regards', 'perfect', 'instants', 'refined', 'coddle', 'fickler', 'gambol', 'misprint', 'tapes', 'impugns', 'moonshot', 'chump', 'brick', 'siren', 'faddish', 'winced', 'kielbasy', 'market', 'puckered', 'trains', 'welts', 'cackles', 'foaming', 'proceed', 'gliding', 'guts', 'uric', 'oaks', 'molested', 'curled', 'boor', 'solaria', 'gristle', 'bombing', 'loamier', 'ensuing', 'cunt', 'sunder', 'revel', 'coaster', 'grunts', 'mucking', 'typesets', 'carnal', 'whimsy', 'scoff', 'coccyxes', 'meanly', 'sprain', 'minuend', 'ringlet', 'fest', 'winced', 'shinier', 'dicier', 'thirds', 'olives', 'garoting', 'pastrami', 'tranquil', 'tamped', 'sunup', 'crumbled', 'throw', 'ridges', 'chaplets', 'curlier', 'lugs', 'collies', 'adapting', 'demeanor', 'deepen', 'lanyard', 'tiller', 'transfix', 'wariness', 'times', 'mitts', 'dowses', 'creels', 'curds', 'quashed', 'orgasmic', 'ibex', 'retraces', 'casino']
ds = random.sample(ds_original, 15)
ds

In [ ]:
ds_original

In [ ]:
print('grid = ', g)
print('padded_grid = ', p)
print('present_words = ', ws)
print('decoy_words = ', ds)
print('Directions: ', [(w, '`' + str(present(g, w)) + '`') for w in ws])

In [ ]:
# with open('example-wordsearch.txt', 'w') as f:
#     f.write('10x10\n')
#     f.write(show_grid(p))
#     f.write('\n')
#     f.write(lcat(sorted(ws + ds)))
# with open('exmaple-wordsearch-solution.txt', 'w') as f:
#     f.write('10x10\n')
#     f.write(show_grid(g))
#     f.write('\n')
#     f.write(lcat(sorted(ws)) + '\n\n')
#     f.write(lcat(sorted(ds)))

In [ ]:
cts = collections.Counter()
for w in ws:
    _, r, c, d = present(g, w)
    inds = indices(g, r, c, len(w), d)
    for i in inds:
        cts[i] += 1
    print(w, r, c, len(w), d, inds)
[i for i in cts if cts[i] > 1]

In [ ]:
# for i in range(100):
#     print(i)
#     g, ws = interesting_grid()
#     p = pad_grid(g)
#     ds = decoys(p, ws, ws_words)
#     with open('wordsearch{:02}.txt'.format(i), 'w') as f:
#         f.write('20x20\n')
#         f.write(show_grid(p))
#         f.write('\n')
#         f.write(lcat(sorted(ws + ds)))
#     with open('wordsearch-solution{:02}.txt'.format(i), 'w') as f:
#         f.write('20x20\n')
#         f.write(show_grid(g))
#         f.write('\n')
#         f.write(lcat(sorted(ws)) + '\n\n')
#         f.write(lcat(sorted(ds)))

In [ ]:
g, ws = interesting_grid(width=100, height=100, words_limit=1, max_words=2000, direction_slack=1)
p = pad_grid(g)
ds = decoys(p, ws, ws_words, limit=2500)
# print(show_grid(g))
# print()
# print(show_grid(p))

In [ ]:
print(len(ws), 'words added; ', len(set(present(g, w)[3] for w in ws)), 'directions')
# print('Present:', wcat(sorted(ws)))
# print('Decoys:', wcat(sorted(ds)))

In [ ]:
with open('huge-wordsearch.txt', 'w') as f:
        f.write('{}x{}\n'.format(len(g[0]), len(g)))
        f.write(show_grid(p))
        f.write('\n')
        f.write(lcat(sorted(ws + ds)))

In [30]:
def read_wordsearch(fn):
    lines = [l.strip() for l in open(fn).readlines()]
    w, h = [int(s) for s in lines[0].split('x')]
    grid = lines[1:h+1]
    words = lines[h+1:]
    return w, h, grid, words

In [31]:
def do_wordsearch_tasks(fn, show_anyway=False):
    width, height, grid, words = read_wordsearch(fn)
    used_words = [w for w in words if present(grid, w)[0]]
    unused_words = [w for w in words if not present(grid, w)[0]]
    lwp = sorted([w for w in words if present(grid, w)[0]], key=len)[-1]
    lwps = [w for w in used_words if len(w) == len(lwp)]
    lwa = sorted(unused_words, key=len)[-1]
    lwas = [w for w in unused_words if len(w) == len(lwa)]
    g0 = empty_grid(width, height)
    for w in words:
        p, r, c, d = present(grid, w)
        if p:
            set_grid(g0, r, c, d, w) 
    unused_letters = [l for l, u in zip((c for c in cat(cat(l) for l in grid)), (c for c in cat(cat(l) for l in g0)))
                  if u == '.']
    unused_letter_count = collections.Counter(unused_letters)
    makeable_words = []
    for w in unused_words:
        unused_word_count = collections.Counter(w)
        if all(unused_word_count[l] <= unused_letter_count[l] for l in unused_word_count):
            makeable_words += [w]
    lwm = sorted(makeable_words, key=len)[-1]
    lwms = [w for w in makeable_words if len(w) == len(lwm)]
    if show_anyway or len(set((len(lwp),len(lwa),len(lwm)))) == 3:
        print('\n{}'.format(fn))
        print('{} words present'.format(len(words) - len(unused_words)))
        print('Longest word present: {}, {} letters ({})'.format(lwp, len(lwp), lwps))
        print('Longest word absent: {}, {} letters ({})'.format(lwa, len(lwa), lwas))
        print('{} unused letters'.format(len([c for c in cat(cat(l) for l in g0) if c == '.'])))
        print('Longest makeable word: {}, {} ({})'.format(lwm, len(lwm), lwms))

In [ ]:
[w for w in ws_words if len(w) == 16]

In [ ]:
# all_lines = []
# for fn in sorted(os.listdir()):
#     if re.match('wordsearch\d\d\.txt', fn):
#         all_lines += open(fn).readlines()
#         all_lines += ['\n'] * 2

# open('all-wordsearches.txt', 'w').writelines(all_lines)

In [209]:
g = empty_grid(10, 10)
g, ws = fill_grid(g, ws_words, 200, min_word_len=4, max_attempts=20)
len(ws)

Added 6 ; attempt 19


6

In [210]:
g = empty_grid(100, 100)
g, ws = fill_grid(g, ws_words, 2000, min_word_len=5)
len(ws)

Added 1149 ; attempt 9427


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.


In [216]:
pm = present_many(g, ws)
pold = [w for w in ws if present(g, w)[0]]
len(pm), len(pold), collections.Counter(p[0] for p in pm).most_common(1)

(1149, 1149, [('scofflaw', 1)])

In [219]:

# with open('huge-unpadded-wordsearch.txt', 'w') as f:
#         f.write('{}x{}\n'.format(len(g[0]), len(g)))
#         f.write(show_grid(g))
#         f.write('\n')
#         f.write(lcat(sorted(ws)))

In [41]:
w, h, g, ws = read_wordsearch('huge-unpadded-wordsearch.txt')
g = [[c for c in row] for row in g]
w, h, len(ws)

(100, 100, 1149)

In [42]:
print(show_grid(g))

smuiclacphysics.hcnebrevilingd..skcabllup.s...gnitnacs..blusted.....pilafeldrig..enquireddetnawnu...
e....nutmegs.t..telbanimret...e..tseikwagewoodpeckerstirwas.b.r...picksn...ylhsiroobsuspectingsexevd
cgnittacdliw..isuoniallivsegas.lrafts.y.vcs..demrofsnart.mla...e.c....e..fannysdiscrimination.oe..e.
nnytima.wharfrep.stswashbucklersw....m.eos...noprat....ao.rs....t.h..m.relabele..sss...b.sc.ts..vb.s
eid..desivedaia.shegdtg.gniyugys.o..a.rme.gpantedsobeysodgm.a....t.osabound...s.bttp...uje.orhd.boe.
tverediw...dtlw.eiar.ehacrierhsrmccosiant.nbestridi..sreepegarotedadr.slaropauses..e..rdis.oteia.lld
nardd.p...itli.i.pesu.srwe...ciuuhfsnti.sgingivitislubl.fotkcuhs..rmsensitisedeu...eleygbutsiyjcf.u.
eerne.u..aiin.r.i.nset.ioku..taeia.gods.e.n..dh.n.emld.r.lht.hobnobsi.s..l.c.trs...jnb.ieoizw.llka.f
shiense.nhae.s.n.aietun.owig.idtrp.srsayi.w..ogu.oecnie..yo.u...whmt.t..u.a.lcds...tb.ondran.oeersa.
.nhpnnktsts...g.d.cve.qe.pblohdoaseomtc.r.a.tgnrgdnaiae.spd.sc.sora...nt.r.en.no..ro...g.l.

In [40]:
padded = False
padding_attempts = 0
while not padded and padding_attempts < 10**6:
    clear_output(wait=True)
    padding_attempts += 1
    pg = pad_grid(g)
    pm = present_many(pg, ws)
    print('Attempt', padding_attempts, '; most common is', collections.Counter(p[0] for p in pm).most_common(1))
    if collections.Counter(p[0] for p in pm).most_common(1)[0][1] == 1:
        padded = True
        
padded, collections.Counter(p[0] for p in pm).most_common(1)

KeyboardInterrupt: 

In [43]:
_, _, pg, _ = read_wordsearch('huge-wordsearch.txt')
pg = [[c for c in row] for row in pg]
print(show_grid(pg))

smuiclacphysicsihcnebrevilingdroskcabllupcsowygnitnacsipblustedpesnipilafeldrigclenquireddetnawnufly
eqdbmnutmegsvtzdtelbanimretanmelptseikwagewoodpeckerstirwaszbdrrdypicksnzikylhsiroobsuspectingsexevd
cgnittacdliwffisuoniallivsegasplraftstyevcsgcdemrofsnartrmlaydzescpopxehsfannysdiscriminationdoeypen
nnytimavwharfrepjstswashbucklerswklqimkeosulxnopratpsitaoorsrtrjtthnsmxrelabelesrssstribiscutsddvbrs
eidgkdesivedaiajshegdtgrgniyugyshohaabrmedgpantedsobeysodgmraoozdteosaboundgfjsmbttpygjujedorhdvboeh
tverediwtczdtlwdeiaryehacrierhsrmccosiantnnbestridikysreepegarotedadrpslaropausesubepnrdissoteiailld
narddypasjitligihpesubsrwewfgciuuhfsntizsgingivitislublefotkcuhschrmsensitisedeugnveleygbutsiyjcflud
eernefunuaiindrsilnsethiokutttaeiapgodsteknerdhlngemldlrslhtrhobnobsizsvdlocdtrszbnjnbpieoizwillkarf
shiensetnhaepslnnaietundowigvidtrptsrsayiewreogudoecniebbyofugamwhmtntwsufaulcdsmlptbeondrancoeersak
onhpnnktstsdovgsdicvevqeypblohdoaseomtcdrbaatgnrgdnaiaejspdtscrsoratzdntmrgenqnomiroixqgsll

In [222]:
padding_attempts

249

In [56]:
dws = decoys(pg, ws, ws_words, limit=2000, min_word_len=5)
len(dws)

851

In [231]:
print(show_grid(pg))

smuiclacphysicsihcnebrevilingdroskcabllupcsowygnitnacsipblustedpesnipilafeldrigclenquireddetnawnufly
eqdbmnutmegsvtzdtelbanimretanmelptseikwagewoodpeckerstirwaszbdrrdypicksnzikylhsiroobsuspectingsexevd
cgnittacdliwffisuoniallivsegasplraftstyevcsgcdemrofsnartrmlaydzescpopxehsfannysdiscriminationdoeypen
nnytimavwharfrepjstswashbucklerswklqimkeosulxnopratpsitaoorsrtrjtthnsmxrelabelesrssstribiscutsddvbrs
eidgkdesivedaiajshegdtgrgniyugyshohaabrmedgpantedsobeysodgmraoozdteosaboundgfjsmbttpygjujedorhdvboeh
tverediwtczdtlwdeiaryehacrierhsrmccosiantnnbestridikysreepegarotedadrpslaropausesubepnrdissoteiailld
narddypasjitligihpesubsrwewfgciuuhfsntizsgingivitislublefotkcuhschrmsensitisedeugnveleygbutsiyjcflud
eernefunuaiindrsilnsethiokutttaeiapgodsteknerdhlngemldlrslhtrhobnobsizsvdlocdtrszbnjnbpieoizwillkarf
shiensetnhaepslnnaietundowigvidtrptsrsayiewreogudoecniebbyofugamwhmtntwsufaulcdsmlptbeondrancoeersak
onhpnnktstsdovgsdicvevqeypblohdoaseomtcdrbaatgnrgdnaiaejspdtscrsoratzdntmrgenqnomiroixqgsll

In [57]:
len(dws) + len(ws)

2000

In [58]:
len(set(dws + ws))

2000

In [59]:
[(a, b) for a in (dws + ws) for b in (dws + ws) if a != b if a in b]

[]

In [60]:
with open('huge-wordsearch.txt', 'w') as f:
        f.write('{}x{}\n'.format(len(pg[0]), len(pg)))
        f.write(show_grid(pg))
        f.write('\n')
        f.write(lcat(sorted(ws + dws)))

In [49]:
pm = present_many(g, ws)
pold = [w for w in ws if present(g, w)[0]]
len(pm), len(pold)

KeyboardInterrupt: 

In [ ]:
ppm = present_many(pg, ws)
ppold = [w for w in ws if present(pg, w)[0]]
len(ppm), len(ppold)

In [236]:
print(show_grid(g))

smuiclacphysics.hcnebrevilingd..skcabllup.s...gnitnacs..blusted.....pilafeldrig..enquireddetnawnu...
e....nutmegs.t..telbanimret...e..tseikwagewoodpeckerstirwas.b.r...picksn...ylhsiroobsuspectingsexevd
cgnittacdliw..isuoniallivsegas.lrafts.y.vcs..demrofsnart.mla...e.c....e..fannysdiscrimination.oe..e.
nnytima.wharfrep.stswashbucklersw....m.eos...noprat....ao.rs....t.h..m.relabele..sss...b.sc.ts..vb.s
eid..desivedaia.shegdtg.gniyugys.o..a.rme.gpantedsobeysodgm.a....t.osabound...s.bttp...uje.orhd.boe.
tverediw...dtlw.eiar.ehacrierhsrmccosiant.nbestridi..sreepegarotedadr.slaropauses..e..rdis.oteia.lld
nardd.p...itli.i.pesu.srwe...ciuuhfsnti.sgingivitislubl.fotkcuhs..rmsensitisedeu...eleygbutsiyjcf.u.
eerne.u..aiin.r.i.nset.ioku..taeia.gods.e.n..dh.n.emld.r.lht.hobnobsi.s..l.c.trs...jnb.ieoizw.llka.f
shiense.nhae.s.n.aietun.owig.idtrp.srsayi.w..ogu.oecnie..yo.u...whmt.t..u.a.lcds...tb.ondran.oeersa.
.nhpnnktsts...g.d.cve.qe.pblohdoaseomtc.r.a.tgnrgdnaiae.spd.sc.sora...nt.r.en.no..ro...g.l.